In [12]:
import joblib
from scipy import sparse
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import hstack

In [ ]:
ingredient_vectorizer = joblib.load("ingredient_vectorizer.joblib")
palette_binarizer = joblib.load("palette_binarizer.joblib")
X_combined = sparse.load_npz("X_combined.npz")
knn = joblib.load("/root/gpu_project/recipeproject/palette/ken/knn_model_palette.joblib")

In [14]:
file_path = '/root/gpu_project/recipeproject/palette/foodrecsys_v1_with_palette.csv'
df = pd.read_csv(file_path)

In [15]:
# ------------------- Recommendation Function -------------------
def recommend_recipes(input_ingredients_str, input_palette_str, top_k=5):
    # Prepare input ingredients
    input_ingredients = ', '.join(sorted([i.strip().lower() for i in input_ingredients_str.split(',')]))
    ingredient_vec = ingredient_vectorizer.transform([input_ingredients])
    
    # Prepare palette
    input_palette = [p.strip().lower() for p in input_palette_str.split(',') if p.strip()]
    palette_vec = palette_binarizer.transform([input_palette])
    
    # Combine sparse features
    input_vec = hstack([ingredient_vec, palette_vec])
    
    # Compute cosine similarity
    similarities = cosine_similarity(input_vec, X_combined).flatten()
    top_indices = similarities.argsort()[::-1][:top_k]
    
    # Optional: Add match score based on exact ingredient overlap
    input_set = set(i.strip().lower() for i in input_ingredients_str.split(','))
    df['match_score'] = df['ingredients_list'].apply(
        lambda ing: len(input_set & set(i.strip().lower() for i in ing.split(',')))
    )
    
    # Select and sort results
    results = df.iloc[top_indices].copy()
    results = results.sort_values(by='match_score', ascending=False)
    return results[['recipe_name', 'ingredients_list', 'palette', 'match_score', 'image_url']]


In [16]:
# -------------------- Example Test --------------------
# input_ingredients = 'pork belly, smoked paprika, kosher salt'
input_ingredients = 'pork belly, smoked paprika, kosher salt'
input_palette = 'fruity, savory'
recommendations = recommend_recipes(input_ingredients, input_palette)
print(recommendations.head(5))

                      recipe_name  \
0                  Homemade Bacon   
7946            Crispy Pork Belly   
10597        Marmalade Pork Chops   
17179  Alyssa's Mango Peach Salsa   
32933                     Espiros   

                                        ingredients_list  \
0      ['pork belly', 'smoked paprika', 'kosher salt'...   
7946   ['pork belly', 'smoked paprika', 'kosher salt ...   
10597    ['pork chops', 'orange marmalade', 'soy sauce']   
17179  ['diced mango', 'diced peaches', 'vegetable oi...   
32933    ['eggs', 'salt', 'orange juice', 'white sugar']   

                     palette  match_score  \
0       savory, smoky, spicy            0   
7946    savory, smoky, spicy            0   
10597  fruity, savory, umami            0   
17179         fruity, savory            0   
32933  fruity, savory, sweet            0   

                                               image_url  
0      https://images.media-allrecipes.com/userphotos...  
7946   https://images.media

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
import numpy as np

# Global variable to track shown indices
shown_indices = set()

def recommend_recipes(input_ingredients_str, input_palette_str, top_k=3, show_more=False):
    global shown_indices

    # Prepare input ingredients
    input_ingredients = ', '.join(sorted([i.strip().lower() for i in input_ingredients_str.split(',')]))
    ingredient_vec = ingredient_vectorizer.transform([input_ingredients])
    
    # Prepare palette
    input_palette = [p.strip().lower() for p in input_palette_str.split(',') if p.strip()]
    palette_vec = palette_binarizer.transform([input_palette])
    
    # Combine sparse features
    input_vec = hstack([ingredient_vec, palette_vec])
    
    # Compute cosine similarity
    similarities = cosine_similarity(input_vec, X_combined).flatten()
    sorted_indices = similarities.argsort()[::-1]

    # Filter out already shown recipes
    new_indices = [i for i in sorted_indices if i not in shown_indices]

    # Take top_k new results
    top_indices = new_indices[:top_k]
    shown_indices.update(top_indices)  # Mark these as shown

    # Compute match score
    input_set = set(i.strip().lower() for i in input_ingredients_str.split(','))
    df['match_score'] = df['ingredients_list'].apply(
        lambda ing: len(input_set & set(i.strip().lower() for i in ing.split(',')))
    )

    # Return sorted recommendations
    results = df.iloc[top_indices].copy()
    results = results.sort_values(by='match_score', ascending=False)
    return results[['recipe_name', 'ingredients_list', 'palette', 'match_score', 'image_url']]


In [18]:
# Initial recommendation
input_ingredients = 'pork belly, smoked paprika, kosher salt'
input_palette = 'spicy, umami'
recommendations = recommend_recipes(input_ingredients, input_palette)
print(recommendations)


             recipe_name                                   ingredients_list  \
0         Homemade Bacon  ['pork belly', 'smoked paprika', 'kosher salt'...   
7946   Crispy Pork Belly  ['pork belly', 'smoked paprika', 'kosher salt ...   
1149  Ground Pork Omelet  ['vegetable oil', 'eggs', 'fish sauce', 'peppe...   

                   palette  match_score  \
0     savory, smoky, spicy            0   
7946  savory, smoky, spicy            0   
1149          spicy, umami            0   

                                              image_url  
0     https://images.media-allrecipes.com/userphotos...  
7946  https://images.media-allrecipes.com/userphotos...  
1149  https://images.media-allrecipes.com/userphotos...  


In [8]:
more_recommendations = recommend_recipes(input_ingredients, input_palette)
print(more_recommendations)

                              recipe_name  \
0                          Homemade Bacon   
7946                    Crispy Pork Belly   
17617  Smokey Parmesan Sweet Potato Chips   
12175                    Hot Dog Sausages   
3567                   Easy Shredded Pork   

                                        ingredients_list  \
0      ['pork belly', 'smoked paprika', 'kosher salt'...   
7946   ['pork belly', 'smoked paprika', 'kosher salt ...   
17617  ['large sweet potato', 'olive oil', 'shaved Pa...   
12175  ['lean ground beef', 'ground pork', 'kosher sa...   
3567   ['paprika', 'garlic powder', 'brown sugar', 'd...   

                                   palette  match_score  \
0                     savory, smoky, spicy            0   
7946                  savory, smoky, spicy            0   
17617  earthy, savory, smoky, spicy, umami            0   
12175                 savory, smoky, spicy            0   
3567                  savory, spicy, sweet            0   

             

In [19]:
more_recommendations = recommend_recipes(input_ingredients, input_palette)
print(more_recommendations)

                              recipe_name  \
17617  Smokey Parmesan Sweet Potato Chips   
40999         Japanese Sesame Steak Sauce   
3185               Kickin Stuffed Peppers   

                                        ingredients_list  \
17617  ['large sweet potato', 'olive oil', 'shaved Pa...   
40999  ['tahini', 'soy sauce', 'mayonnaise', 'garlic'...   
3185   ['mild pork sausage', 'Neufchatel cheese', 'fr...   

                                   palette  match_score  \
17617  earthy, savory, smoky, spicy, umami            0   
40999                 savory, spicy, umami            0   
3185                          spicy, umami            0   

                                               image_url  
17617  http://images.media-allrecipes.com/userphotos/...  
40999  https://images.media-allrecipes.com/userphotos...  
3185   https://images.media-allrecipes.com/userphotos...  


In [20]:
more_recommendations = recommend_recipes(input_ingredients, input_palette)
print(more_recommendations)

                 recipe_name  \
31857  Crispy Turnip 'Fries'   
28121   Easy Pizza Sauce III   
2743        Yummy Pork Chops   

                                        ingredients_list  \
31857  ['turnips', 'vegetable oil', 'grated Parmesan ...   
28121  ['tomato sauce', 'tomato paste', 'ground orega...   
2743   ['Italian-style salad dressing', 'soy sauce', ...   

                           palette  match_score  \
31857         savory, spicy, umami            0   
28121  herby, savory, spicy, umami            0   
2743          savory, spicy, umami            0   

                                               image_url  
31857  https://images.media-allrecipes.com/userphotos...  
28121  https://images.media-allrecipes.com/userphotos...  
2743   https://images.media-allrecipes.com/userphotos...  
